In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.probability import FreqDist
import lime.lime_text

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/uu_ics_ads/sverkleij/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from collections import defaultdict
from pathlib import Path
import os
import csv
import pandas as pd
import fasttext
main_folder = 'Data/MD'

In [4]:
from nltk.corpus import stopwords
stop = stopwords.words('dutch')

In [3]:
results = defaultdict(list)
for file in Path(main_folder).iterdir():
    with open(file, encoding = "ISO-8859-1") as file_open:
        results["file_name"].append(file.name)
        results["text"].append(file_open.read())
        
df = pd.DataFrame(results)

In [4]:
results = defaultdict(list)
for file in Path('Data/NoMD').iterdir():
    with open(file, encoding = "ISO-8859-1") as file_open:
        results["file_name"].append(file.name)
        results["text"].append(file_open.read())
        
df2 = pd.DataFrame(results)

In [5]:
df2['Disorder']='nomd'
df['Disorder']='md'
result1 = pd.concat([df, df2])
del result1['file_name']

In [6]:
result1['Text'] = result1['text']
del result1['text']

In [12]:
result1['Text'] = result1['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [7]:
dfmentald = pd.read_excel(r'MentalDisorders/LIWCresultstotal.xlsx')
df.columns = ['Filename','Text']
result = pd.merge(df, dfmentald, on='Filename', how = 'outer').fillna(0)
del result['Filename']
result1 = result

In [9]:
result1['Text']=result1['Text'].str.replace(r',', '')

In [ ]:
from sklearn.model_selection import train_test_split
# Train Test Split Function
def split_train_test(result, test_size=0.2, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(result1[['Text']], 
                                                       result1['Disorder'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(result1)

In [11]:
result2 = pd.merge(X_train, Y_train, on='index', how = 'outer').fillna(0)

In [ ]:
col = ['Disorder', 'Text']

yelp_reviews = result2[col]
yelp_reviews['Disorder']=['__label__'+ s for s in yelp_reviews['Disorder']]
yelp_reviews['Text']= yelp_reviews['Text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
yelp_reviews.to_csv(r'MentalDisorders/yelp_reviews_updated.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [ ]:
resulttest = pd.merge(X_test, Y_test, on='index', how = 'outer').fillna(0)

In [ ]:
col = ["Disorder", "Text"]
testset = resulttest[col]
testset['Disorder']=['__label__'+ s for s in testset['Disorder']]
testset['Text']= testset['Text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
testset.to_csv(r'MentalDisorders/yelp_test.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [15]:
model = fasttext.train_supervised("MentalDisorders/yelp_reviews_updated.txt", epoch=500)

In [ ]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))
                                
print_results(*model.test("MentalDisorders/yelp_test.txt"))

In [ ]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score

# predict the data
testset["predicted"] = testset["Text"].apply(lambda x: model.predict(x)[0][0])

# Create the confusion matrix
confusion_matrix(testset["Disorder"], testset["predicted"])



In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
cohen_kappa_score(testset['Disorder'], testset['predicted'])

In [ ]:
accuracy_score(testset['Disorder'], testset['predicted'])

In [20]:
#import fastText
import re
import webbrowser

def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

def tokenize_string(string):
    return string.split()

classifier = (model)

explainer = lime.lime_text.LimeTextExplainer(
    split_expression=tokenize_string,
    bow=False,
    class_names=["__label__nomd", "__label__md"]
)

def fasttext_prediction_in_sklearn_format(classifier, texts):
    res = []
    labels, probabilities = classifier.predict(texts, 10)
    for label, probs, text in zip(labels, probabilities, texts):
        order = np.argsort(np.array(label))
        res.append(probs[order])

    return np.array(res)


In [28]:
review = testset.iloc[4,1]

In [ ]:
# Pre-process the text of the review so it matches the training format
preprocessed_review = strip_formatting(review)

exp = explainer.explain_instance(
    preprocessed_review,
    classifier_fn=lambda x: fasttext_prediction_in_sklearn_format(classifier, x),
    top_labels=2,
    num_features=10,
)

exp.show_in_notebook(text=preprocessed_review)

In [ ]:
exp.as_list()

In [ ]:
results = (model.predict(review))
print (results[1][0])
print(testset.iloc[1,0])

In [ ]:
%matplotlib inline
fig = exp.as_pyplot_figure()